In [1]:
import yake
from transformers import BertTokenizer, BertModel
import pandas as pd

/Users/jonathanhu/anaconda3/envs/tf_metal/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option("display.max_colwidth", None)

# Load Yelp reviews data
reviews_df = pd.read_csv('yelp reviews.csv')

# Display the first few rows of the dataframe
reviews_df.head()

,Restaurant name,Restaurant url,User,User location,Number of friends,Number of reviews,Raitng,Date,Comment,Useful,Funny,Cool,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18
0,$1 Noodle Cart,https://www.yelp.com/biz/1-noodle-cart-new-york,Chris F.,"Brooklyn, NY",54,246,3,7/11/2005,"This street cart -- vaguely where 147 Centre St would be were that a real address -- serves up super greasy noodles, fried rice, egg rolls, and other such cheap fare for the shocking price of $1. The quality is about what you'd expect for a $1 container of fried food, but it's a reliable lunch option, and always has a decent line around noon.",1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,$1 Noodle Cart,https://www.yelp.com/biz/1-noodle-cart-new-york,Vilay M.,"Minneapolis, MN",17,14,3,11/15/2008,What would you expect from a cart? Lots of food! For $1! I got the noodles of course. For a dollar it was good and greasy. If only she spoke english...,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1 Chimi Sushi,https://www.yelp.com/biz/1-chimi-sushi-new-york,Allison P.,"Stuart, FL",97,279,1,5/16/2011,"very bland rolls...no spice whatsoever in the spicy tuna roll, and no jalapeo spice in the green river roll...yellowtail scallion roll had no scallions! i've had better sushi from the deli at the grocery store. also, delivery was SLOW...i was quoted 25-40 mins, and it took 1:15.",1.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1 Chimi Sushi,https://www.yelp.com/biz/1-chimi-sushi-new-york,Amy T.,"New York, NY",0,16,3,10/8/2011,"Can't say anything about their rolls because I didn't get it. I had eel over rice. Their potion is very small, not worth $10 but the rice was very flavorful. My friend had the chicken katsu and it was twice my dish size so I end up eating some of her food. I gave them 3 stars instead of 2 is because student gets 10% discount.",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1 Chimi Sushi,https://www.yelp.com/biz/1-chimi-sushi-new-york,Christina W.,"Little Neck, NY",0,40,5,4/29/2011,"I love this place. I'm so shocked that nobody yelped this place yet. I actually found them because I was searching Seamless Web which is a website that tells you the restaurants that deliver around your neighborhood. SO actually, I haven't been there yet but i order from them all the time and they are great. The food always taste fresh and the price, you can't beat it. They are also very good with service and they try to accommodate you in every way. Just try them once and you won't be disappointed at the food. I really love the spicy scallops. YUMMY TO MY TUMMY!!!!!!!",2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import re

def preprocess_text(text):
    """
    Preprocess the text data by lowercasing, removing special characters,
    and optionally removing stopwords.
    """
    # Lowercase the text
    text = text.lower()
    # Remove special characters
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    # Optionally remove stopwords
    text = ' '.join([word for word in text.split() if word not in ENGLISH_STOP_WORDS])
    return text

# Apply preprocessing to the Comment column
reviews_df['Processed_Comment'] = reviews_df['Comment'].astype(str).apply(preprocess_text)

# Display the first few rows of the processed dataframe
reviews_df[['Comment', 'Processed_Comment']].head()


,Comment,Processed_Comment
0,"This street cart -- vaguely where 147 Centre St would be were that a real address -- serves up super greasy noodles, fried rice, egg rolls, and other such cheap fare for the shocking price of $1. The quality is about what you'd expect for a $1 container of fried food, but it's a reliable lunch option, and always has a decent line around noon.",street cart vaguely 147 centre st real address serves super greasy noodles fried rice egg rolls cheap fare shocking price 1 quality youd expect 1 container fried food reliable lunch option decent line noon
1,What would you expect from a cart? Lots of food! For $1! I got the noodles of course. For a dollar it was good and greasy. If only she spoke english...,expect cart lots food 1 got noodles course dollar good greasy spoke english
2,"very bland rolls...no spice whatsoever in the spicy tuna roll, and no jalapeo spice in the green river roll...yellowtail scallion roll had no scallions! i've had better sushi from the deli at the grocery store. also, delivery was SLOW...i was quoted 25-40 mins, and it took 1:15.",bland rollsno spice whatsoever spicy tuna roll jalapeo spice green river rollyellowtail scallion roll scallions ive better sushi deli grocery store delivery slowi quoted 2540 mins took 115
3,"Can't say anything about their rolls because I didn't get it. I had eel over rice. Their potion is very small, not worth $10 but the rice was very flavorful. My friend had the chicken katsu and it was twice my dish size so I end up eating some of her food. I gave them 3 stars instead of 2 is because student gets 10% discount.",say rolls didnt eel rice potion small worth 10 rice flavorful friend chicken katsu twice dish size end eating food gave 3 stars instead 2 student gets 10 discount
4,"I love this place. I'm so shocked that nobody yelped this place yet. I actually found them because I was searching Seamless Web which is a website that tells you the restaurants that deliver around your neighborhood. SO actually, I haven't been there yet but i order from them all the time and they are great. The food always taste fresh and the price, you can't beat it. They are also very good with service and they try to accommodate you in every way. Just try them once and you won't be disappointed at the food. I really love the spicy scallops. YUMMY TO MY TUMMY!!!!!!!",love place im shocked yelped place actually searching seamless web website tells restaurants deliver neighborhood actually havent order time great food taste fresh price beat good service try accommodate way just try wont disappointed food really love spicy scallops yummy tummy


We'll work with the following approaches:

### TF-IDF (Term Frequency-Inverse Document Frequency): 
This method will help us identify important words in each review by considering how frequently they appear in a particular review relative to their frequency across all reviews.

### LDA (Latent Dirichlet Allocation): 
This is a topic modeling approach that will help us identify topics within the reviews, which can indirectly point us to keywords.

For each model, we'll extract keywords or key phrases from the reviews. Then, we'll compare their performance qualitatively by examining the relevance and quality of the extracted keywords.

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def extract_keywords_tfidf(docs, n_keywords=5):
    """
    Extract keywords from documents using TF-IDF
    :param docs: List of documents (preprocessed reviews)
    :param n_keywords: Number of top keywords to extract from each document
    :return: List of keywords for each document
    """
    vectorizer = TfidfVectorizer(max_df=0.85, max_features=1000)
    tfidf_matrix = vectorizer.fit_transform(docs)
    feature_names = np.array(vectorizer.get_feature_names_out())

    keywords_list = []
    for row in tfidf_matrix:
        sorted_indices = row.toarray().flatten().argsort()[-n_keywords:][::-1]
        keywords = feature_names[sorted_indices]
        keywords_list.append(keywords)

    return keywords_list

# Extract the comments for keyword extraction
comments = reviews_df['Processed_Comment'].dropna().tolist()

# Extract keywords using TF-IDF
tfidf_keywords = extract_keywords_tfidf(comments)

# Displaying the first few sets of keywords
tfidf_keywords[:5]


[array(['fried', 'st', 'youd', 'greasy', 'option'], dtype=object),
 array(['dollar', 'greasy', 'lots', 'noodles', 'expect'], dtype=object),
 array(['spice', 'roll', 'mins', 'deli', 'store'], dtype=object),
 array(['10', 'rice', 'twice', 'gets', 'size'], dtype=object),
 array(['actually', 'love', 'try', 'seamless', 'deliver'], dtype=object)]

### NOT IDEAL
The above model takes around 4 seconds to finish, given the model size is around 111,000 rows. It is clear from the above that this model is not ideal since it only takes into account the frequency that the words appear, both in one certain review and across all datasets. Let's try something else. 

In [5]:
# Initialize YAKE
kw_extractor = yake.KeywordExtractor()

# Function to extract keywords
def extract_keywords(text):
    keywords = kw_extractor.extract_keywords(text)
    return [kw[0] for kw in keywords][:5]  # top 5 keywords

# Apply keyword extraction to the reviews
reviews_df_sample = reviews_df[:10]
reviews_df_sample['Keywords'] = reviews_df_sample['Comment'].apply(extract_keywords)

# Display the updated DataFrame
reviews_df_sample['Keywords']


/var/folders/v3/hwmt_9nx6nl41f06plb5szqr0000gn/T/ipykernel_11084/2547752933.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df_sample['Keywords'] = reviews_df_sample['Comment'].apply(extract_keywords)


0                     [super greasy noodles, egg rolls, street cart, real address, serves up super]
1                                                          [cart, Lots of food, expect, Lots, food]
2    [yellowtail scallion roll, spicy tuna roll, green river roll, yellowtail scallion, spicy tuna]
3                                                               [rolls, rice, discount, eel, small]
4              [searching Seamless Web, Seamless Web, yelped this place, place, searching Seamless]
5                               [Ordered the Chirashi, Chirashi, Ordered, average tasting, tasting]
6                     [chimi cocoa pop, cocoa pop, chimi chimi rock, Chimi chimi cocoa, chimi rock]
7                                   [stars on seamless, seamless web, dinner bento box, stars, web]
8     [Sushi Dumpling house, Sushi Dumpling, Dumpling house, previous incarnation, decided to give]
9                                                   [called Chimi, TERRIBLE, Chimi, minutes, order]


In [6]:
reviews_df_sample['Keywords'] = reviews_df_sample['Processed_Comment'].apply(extract_keywords)

# Display the updated DataFrame
reviews_df_sample['Keywords']

/var/folders/v3/hwmt_9nx6nl41f06plb5szqr0000gn/T/ipykernel_11084/3453395577.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df_sample['Keywords'] = reviews_df_sample['Processed_Comment'].apply(extract_keywords)


0                      [street cart vaguely, quality youd expect, fare shocking price, decent line noon, container fried food]
1                              [expect cart lots, cart lots food, greasy spoke english, dollar good greasy, good greasy spoke]
2          [delivery slowi quoted, bland rollsno spice, whatsoever spicy tuna, green river rollyellowtail, sushi deli grocery]
3                      [potion small worth, eating food gave, rolls didnt eel, flavorful friend chicken, friend chicken katsu]
4    [scallops yummy tummy, searching seamless web, seamless web website, restaurants deliver neighborhood, havent order time]
5               [ordered chirashi average, chirashi average tasting, average tasting bad, tasting bad worth, ordered chirashi]
6                                  [spicy tempura tuna, adage true pay, low prices menu, soup edaname salad, scored miso soup]
7         [stars seamless web, dinner bento box, pieces nigiri california, blocks north avenues, avenues restau

Yake offers a better extraction with more context. But maybe we can still do better. We can train BERT models and use attention scores. 